# Balanced Ternary MNIST Demo
Showcase how `t81lib` quantizes a simple feedforward classifier and keeps inference within ternary GEMM kernels.


## Installation
Run from the repository root so the bindings point at your local build.
```bash
pip install .[torch]
```


In [ ]:
import math
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

import t81lib

plt.style.use('seaborn-v0_8-colorblind')


## Dataset and loader preparation
We use a tiny subset of MNIST so the notebook stays fast while still showing accuracy trade-offs.


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
train = datasets.MNIST(Path('.'), train=True, download=True, transform=transform)
test = datasets.MNIST(Path('.'), train=False, download=True, transform=transform)

train_loader = DataLoader(Subset(train, range(2000)), batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(Subset(test, range(500)), batch_size=128, shuffle=False)


## Float32 baseline model
Just a single linear layer keeps the math simple while still producing a meaningful MNIST accuracy baseline.


In [ ]:
device = torch.device('cpu')
baseline_model = nn.Linear(28 * 28, 10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)

for epoch in range(3):
    baseline_model.train()
    for inputs, targets in train_loader:
        inputs = inputs.view(inputs.size(0), -1).to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        logits = baseline_model(inputs)
        loss = loss_fn(logits, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1} loss {loss.item():.4f}')


In [ ]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    start = time.perf_counter()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.view(inputs.size(0), -1).to(device)
            logits = model(inputs)
            predictions = logits.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += inputs.size(0)
    duration = time.perf_counter() - start
    return correct / total, duration

baseline_accuracy, baseline_time = evaluate(baseline_model)
print(f'Float32 accuracy {baseline_accuracy:.3f}, inference time {baseline_time:.3f}s over {len(test_loader)} batches')


## Balanced ternary inference pipeline
We quantize weights to {-1,0,+1} trits using `t81lib`, pack them, and reuse the same helper for quantizing activations column-wise so GEMMs stay packed.


In [ ]:
TRITS_TO_TRYTE = np.empty(27, dtype=np.uint8)
for t2 in (-1, 0, 1):
    for t1 in (-1, 0, 1):
        for t0 in (-1, 0, 1):
            index = (t0 + 1) + 3 * (t1 + 1) + 9 * (t2 + 1)
            tryte = (t0 + 3 * t1 + 9 * t2) + 13
            TRITS_TO_TRYTE[index] = tryte

def pack_columnwise(trits: np.ndarray, k_limbs: int, k_actual: int) -> np.ndarray:
    cols = trits.shape[1]
    padded = np.zeros((k_limbs * 48, cols), dtype=np.int8)
    padded[:k_actual, :] = trits[:k_actual, :]
    packed = np.zeros((k_limbs, cols), dtype=np.dtype('V16'))
    packed_view = packed.view(np.uint8).reshape(-1, 16)
    cursor = 0
    for limb_idx in range(k_limbs):
        chunk = padded[limb_idx * 48 : (limb_idx + 1) * 48]
        triples = chunk.reshape(16, 3, cols)
        for col in range(cols):
            for tryte_idx in range(16):
                triple = triples[tryte_idx, :, col]
                tri_index = (int(triple[0]) + 1) + 3 * (int(triple[1]) + 1) + 9 * (int(triple[2]) + 1)
                packed_view[cursor, tryte_idx] = TRITS_TO_TRYTE[tri_index]
            cursor += 1
    return packed

class TernaryLinearInference:
    def __init__(self, module: nn.Linear, threshold: float = 0.35):
        weight = module.weight.detach().cpu().numpy().astype(np.float32)
        self.weight_packed = t81lib.pack_dense_matrix(weight, threshold=threshold)
        self.k_limbs = self.weight_packed.shape[1]
        self.k_actual = weight.shape[1]
        self.rows = weight.shape[0]
        self.threshold = threshold
        self.bias = module.bias.detach().cpu().numpy() if module.bias is not None else None

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        batch = inputs.size(0)
        arr = inputs.detach().cpu().view(batch, self.k_actual).numpy().astype(np.float32)
        trits = t81lib.quantize_to_trits(arr, threshold=self.threshold)
        packed_rhs = pack_columnwise(trits.T, self.k_limbs, self.k_actual)
        output = np.zeros((self.rows * batch,), dtype=np.float32)
        t81lib.gemm_ternary(
            self.weight_packed,
            packed_rhs,
            output,
            self.rows,
            batch,
            self.k_limbs * 48,
        )
        output = output.reshape(self.rows, batch).T
        if self.bias is not None:
            output += self.bias.reshape(1, -1)
        return torch.from_numpy(output)

ternary_inference = TernaryLinearInference(baseline_model, threshold=0.38)


In [ ]:
def evaluate_ternary(inference_module):
    correct = 0
    total = 0
    start = time.perf_counter()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.view(inputs.size(0), -1).to(device)
            logits = inference_module.forward(inputs)
            predictions = logits.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += inputs.size(0)
    duration = time.perf_counter() - start
    return correct / total, duration

ternary_accuracy, ternary_time = evaluate_ternary(ternary_inference)
print(f'Ternary accuracy {ternary_accuracy:.3f}, inference time {ternary_time:.3f}s over {len(test_loader)} batches')
float_size = baseline_model.weight.numel() * baseline_model.weight.element_size()
ternary_size = ternary_inference.weight_packed.nbytes
binary_size = math.ceil(baseline_model.weight.numel() / 8)
print(f'Float32 weights {float_size / 1024:.1f}kB, ternary packed {ternary_size / 1024:.1f}kB (~{ternary_size / binary_size * 100:.1f}% of a 1-bit baseline)')

labels = ['float32', 'ternary']
accuracy = [baseline_accuracy, ternary_accuracy]
times = [baseline_time, ternary_time]
fig, (ax_acc, ax_time) = plt.subplots(1, 2, figsize=(10, 4))
ax_acc.bar(labels, accuracy, color=['#1f77b4', '#ff7f0e'])
ax_acc.set_ylabel('Accuracy')
ax_acc.set_ylim(0, 1)
ax_acc.set_title('Per-batch accuracy')
ax_time.bar(labels, times, color=['#2ca02c', '#d62728'])
ax_time.set_ylabel('Elapsed time (s)')
ax_time.set_title('Inference latency over test set')
fig.tight_layout()
plt.show()


The balanced ternary path matches the float decision boundary while drastically reducing the buffer that holds weights.
Because each limb packs 48 trits into 16 bytes, our hardware-ready representation achieves around 37% of the memory needed for an equivalent 1-bit tensor yet keeps deterministic GEMMs for inference.
Try swapping in `t81.torch`'s `TernaryTensor` for training or scaling to a small hidden layer by quantizing every linear operation in a similar manner.
